# Optimierung von neuronalen Netzwerken


## Erstellen und Konfigurieren eines neuronalen Netzes

Wie gewohnt importieren wir zuerst alle Module, die wir in unserem Programm brauchen werden.

In [ ]:
# Importiere die nötigen Module
import seaborn as sns
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
import keras

In diesem Jupyter Notebook arbeiten wir wieder mit dem <a href="https://www.kaggle.com/competitions/titanic/overview">_Titanic_</a> Datensatz, mit dessen Hilfe wir ein künstliches neuronales Netz zur Vorhersage der Überlebenschance der Passagiere erstellen und trainieren wollen.

Nachdem wir den Datensatz geladen haben, nehmen wir einige Vorverarbeitungsschritte an diesem vor und teilen ihn in die Training- und die Testmenge auf.


In [ ]:
# Lade den Titanic-Datensatz
titanic = sns.load_dataset("titanic")

# Speichere das zweite (Passagierklasse), das dritte (Geschlecht),
# das fünfte (Anzahl der Geschwister/Ehepartner an Bord), das sechste (Anzahl der Eltern/Kinder an Bord),
# das siebte (Passagiertarif (Britisches Pfund)),
# und das zehnte (Mann/Frau/Kind) Merkmal in die Datenmatrix
X = titanic[['pclass', 'sex', 'sibsp', 'parch', 'fare', 'who']]

# Speichere das erste Merkmal (überlebt) als Zielmerkmal
y = titanic[['survived']]

# Damit unterdrücken wir einige Warnungen
# (Empfehlung von Pandas)
pd.options.mode.copy_on_write = True

# Speichere im Merkmal 'who' nur die Information darüber, ob ein Passagier
# ein Kind war, und benenne es in 'is_child' um
X['who'] = np.where(X['who'] == 'child', 1, 0)
X = X.rename(columns={'who': 'is_child'})

# Umwandlung kategorischer Merkmale nach der One-Hot-Methode
X = pd.get_dummies(X, columns=['sex'], dtype=float)
y = pd.get_dummies(y, columns=['survived'], dtype=float)

# Normalisiere die Merkmalswerte auf den Bereich [0,1]
X_normalized = preprocessing.minmax_scale(X)

# Teile den Datensatz in die Trainings- und die Testmenge auf
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, test_size=0.15, random_state=61)

Wir implementieren zuerst ein sequenzielles Feed-Forward-Netz mit sieben Neuronen in der Eingabeschicht, die den sieben Merkmalen unseres Datensatzes entsprechen. Diese verknüpfen wir mit einer vollständig verbundenen Schicht mit fünf Neuronen und der _ReLU_-Aktivierungsfunktion. Die Ausgabeschicht unseres Netzes enthält zwei Neuronen, die den beiden Klassen _überlebt_ und _nicht überlebt_ entsprechen. Als Aktivierungsfunktion fungiert die _softmax_-Funktion. In der Zusammenfassung sehen wir, dass unser Modell nur 52 Optimierungsparameter hat.

In [ ]:
# Aufbau eines neuronalen Netzes
model = keras.Sequential()
model.add(keras.Input(shape=(X_normalized.shape[1],)))
model.add(keras.layers.Dense(units=5, activation="relu"))
model.add(keras.layers.Dense(2, activation = "softmax"))

model.summary()

Bevor wir unser Modell trainieren können, müssen wir es noch konfigurieren. Die Methode dafür heißt zwar _compile()_, führt aber eher eine Konfiguration des Modells durch, bei der drei wichtige Einstellungen gesetzt werden. Als muss die Verlustfunktion angeben werden, die den Fehler zwischen den berechneten und den tatsächlichen Werten für das Zielmerkmal auf den Trainingsdaten berechnet. Für die Klassifikationsaufgaben mit zwei Klassen entscheidet man sich meistens für die _binary_crossentropy_. Als zweites müssen wir uns auf einen Optimierer festlegen. Dieser bestimmt, wie jeder Optimierungsparameter, also jedes Gewicht und jeder Bias-Term im Rahmen der Backpropagation angepasst wird, um den Fehler zu reduzieren. Wir entscheiden uns für den prominenten _Adamax_-Optimierer. Optional kann man noch eine Liste der Metriken angeben, die während des Trainings neben den Werten für die Verlustfunktion zusätzlich berechnet werden sollen. Wir sind an der Überwachung der _Accuracy_, also der Korrektklassifikationsrate, interessiert.

In [ ]:
# Konfiguriere das Modell
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adamax(learning_rate= 0.001),
              metrics=['accuracy'])

Bei der Konfiguration des Modells werden die Optimierungsparameter mit zufälligen Werten initialisiert. Also eigentlich können mit dem Modell schon Vorhersagen erstellt werden, die natürlich nicht sinnvoll sein müssen, weil das Modell noch nicht trainiert wurde. Die Vorhersagen werden mit der Methode _predict()_ erstellt. Dieser übergeben wir die ersten fünf Beobachtungen des Testdatensatzes und vergleichen die berechneten Ergebnisse mit den tatsächlichen Werten des Zielmerkmals Überlebensstatus.

In [ ]:
# Erstelle eine Vorhersage für die ersten fünf Beobachtungen des Trainingdatensatzes
# mit untrainiertem Modell
print(model.predict(X_test[:5]))
print(y_test[:5])

## Training und Evaluation eines neuronalen Netzes

Nachdem uns die Vorhersagen des untrainierten Modells nicht ganz überzeugt haben, trainieren wir unser neuronales Netzwerk anhand der Trainingsdaten. Dafür müssen wir einfach die Methode _fit()_ aufrufen. Dieser übergeben wir die Eingabemerkmale _X_train_ und die Zielmerkmale _y_train_. Außerdem müssen wir die Anzahl der _Epochen_ und die _batch_size_ angeben. Bei der _batch size_ handelt es sich um die Anzahl der Beobachtungen des Trainingssatzes, nach deren Verarbeitung die Optimierungsparameter des Modells neuberechnet werden. Ein guter Wert hierfür ist 32. Mit der Anzahl der Epochen legen wir fest, wie viel Mal unser neuronales Netz den gesamten Trainingsdatensatz bei dem Trainingsvorgang verarbeiten soll. Wir wollen nicht lange warten, deswegen setzen wir die Anzahl der Epochen auf fünf. Alle weiteren Parameter sind optional. Mit _validation_data_ können die Validierungsdaten übergeben werden, für die die Werte der Verlustfunktion und der Evaluationsmetriken beim Trainingsvorgang vom Modell mitberechnet werden, obwohl diese beim Training selbst nicht benutzt werden. Mit dem optionalen Parameter _verbose_ können wir festlegen, wie die Ausgabe des Trainingsvorgangs aussehen soll.

Die Methode _fit()_ liefert eine _Historie_ zurück, die unter anderem die Werte der Verlustfunktion und der optionalen Bewertungsmetriken vom Ende jeder Epoche für den Trainings- und den Validierungsdatensatz enthält.

In [ ]:
# Hier findet das eigentliche Training des Modells statt
history = model.fit(x=X_train, y=y_train, epochs=5, batch_size=32,
                    validation_data=(X_test, y_test), verbose=2)

Wenn du die Ergebnisse des Trainingsvorgangs lieber in einem Graphen betrachten möchtest, kannst du die Historie als ein DataFrame speichern und mithilfe des _plot()_-Befehls als Lernkurven anzeigen lassen.

In [ ]:
# Zeige die Ergebnisse für Loss und Accuracy pro Epoche in einem Graphen dar
fig = pd.DataFrame(history.history).plot(figsize=(8, 5))

## Unter- und Überanpassung

Wenn die Accuracy-Werte sowohl für den Trainings- als auch den Validierungsdatensatz sehr niedrig sind, liegt es wahrscheinlich daran, dass das Modell zu einfach ist, um die in den Trainingsdaten enthaltenen Strukturen zu lernen. Wenn das auftritt, spricht man auch von _Underfitting_ bzw. _Unteranpassung_ des Modells. Um der Unteranpassung entgegen zu wirken, kann man zum Beispiel ein größeres Modell mit mehr Neuronen bzw. Optimierungsparametern verwenden und es länger trainieren.

In [ ]:
# Aufbau eines neuronalen Netzes
model_2 = keras.Sequential()
model_2.add(keras.Input(shape=(X_normalized.shape[1],)))
model_2.add(keras.layers.Dense(units=15, activation="relu"))
model_2.add(keras.layers.Dense(2, activation = "softmax"))

model_2.summary()

In [ ]:
# Konfiguriere das Modell
model_2.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adamax(learning_rate= 0.001),
              metrics=['accuracy'])

In [ ]:
# Hier findet das eigentliche Training des Modells statt
history_2 = model_2.fit(x=X_train, y=y_train, epochs=100, batch_size=32,
                    validation_data=(X_test, y_test), verbose=2)

In [ ]:
# Zeige die Ergebnisse für Loss und Accuracy pro Epoche in einem Graphen dar
fig_2 = pd.DataFrame(history_2.history).plot(figsize=(8, 5))

Unser Modell overfittet nicht, denn beim _Overfitting_ bzw. _Überanpassung_ würde die Accuracy für den Trainingsdatensatz mit der steigenden Epochenanzahl weiter ansteigen während sie für den Validierungsdatensatz entweder konstant bleiben oder sogar fallen würde. Dazu kommt es meistens, wenn das neuronale Netz zu mächtig ist bzw. zu viele Optimierungsparameter besitzt, die es dem Modell erlauben, sich an die zufälligen Schwankungen in den Trainingsdaten anzupassen, so dass das Modell nicht mehr generalisiert.